In [1]:
%run /eos/project/l/liu/Toolbox/myToolbox.py

%config InlineBackend.figure_format = 'retina'

plt.rcParams["mathtext.fontset"] = "cm"

import csv

Version 0.03. This is the latest version.
Please help me to improve it reporting bugs to guido.sterbini@cern.ch.


/eos/project/l/liu/Toolbox/myToolbox.py:24: FutureWarning: 'pandas.tseries.converter.register' has been moved and renamed to 'pandas.plotting.register_matplotlib_converters'. 
  converter.register()


Your platform is Linux-3.10.0-957.21.3.el7.x86_64-x86_64-with-centos-7.6.1810-Core
Your folder is /eos/user/a/ahuschau/www/test-acc-models/repository/PS/2019/scenarios/SFTPRO/4_horizontal_splitting
Your IP is 172.17.0.4
2019-08-14 09:14:36


# Transfer functions (to be checked with Massimo)

In [23]:
def normalized_gradient(current, magnet):
    Brho = 3.3356 * 14

    if 'XNO' in magnet:
        # integrated field at peak current from NORMA
        GL = 50
        # peak current from NORMA
        Ipeak = 720
        # magnetic length from MTE design report
        lmag = 0.195
        # account for polarity accroding to old MAD-X description
        polarity = -1
        # calculate normalized sextupole gradient
        return polarity * current * GL / Ipeak / lmag / Brho
    elif 'ONO' in magnet:
        # field at nominal current from NORMA
        G = 14360
        # nominal current from NORMA
        Inom = 700
        # account for polarity accroding to old MAD-X description
        polarity = -1
        # calculate normalized octupole gradient
        return polarity * current * G / Inom / Brho
    elif 'ODN' in magnet:
        # integrated field at 100 A from MTE design report
        GL = 433
        # reference current
        I = 100
        # magnetic length from MTE design report
        lmag = 0.22
        # account for polarity accroding to old MAD-X description
        polarity = -1
        # calculate normalized octupole gradient
        return polarity * current * GL / I / lmag / Brho    


# Import of LSA settings

In [27]:
if 1:
    file_nonlin = './non-linear_elements_str.xlsx'
    file_LEQ = './LEQ_str.xlsx'
    LSA_nonlin = pnd.read_excel(file_nonlin)
    LSA_LEQ = pnd.read_excel(file_LEQ)
    LSA_LEQ.set_index('time', inplace = True)

    t = np.arange(700, 851, 1)

    MTE_elements = pnd.DataFrame(columns = ['IXNO39', 'IXNO55', 'IONO39', 'IONO55', 'IODN', 'Qx', 'kQD', 'kQF'], index = t)

    # sextupoles
    MTE_elements['IXNO39'] = np.interp(t, LSA_nonlin['XNO39'], LSA_nonlin['Unnamed: 1'])
    MTE_elements['IXNO55'] = np.interp(t, LSA_nonlin['XNO55'], LSA_nonlin['Unnamed: 3'])

    # octupoles
    MTE_elements['IONO39'] = np.interp(t, LSA_nonlin['ONO39'], LSA_nonlin['Unnamed: 5'])
    MTE_elements['IONO55'] = np.interp(t, LSA_nonlin['ONO55'], LSA_nonlin['Unnamed: 7'])
    MTE_elements['IODN'] = np.interp(t, LSA_nonlin['ODN'], LSA_nonlin['Unnamed: 9'])

    for q in ['Qx','kQD', 'kQF']:
        MTE_elements[q] = LSA_LEQ[q]
    
    # assume that Qx = 6.25 at resonance crossing and calculate the variation along the cycle
    MTE_elements['Qx'] = MTE_elements['Qx'] - MTE_elements['Qx'].loc[720] + 6.25
    
    for c in MTE_elements.columns[:-3]:
        MTE_elements['K' + c[1:]] = MTE_elements[c].apply(lambda x: normalized_gradient(x, c))
        
    MTE_elements.to_pickle('Strengths_of_MTE_elements.pkl')
else:
    MTE_elements = pnd.read_pickle('Strengths_of_MTE_elements.pkl')

In [28]:
MTE_elements.head()

,IXNO39,IXNO55,IONO39,IONO55,IODN,Qx,kQD,kQF,KXNO39,KXNO55,KONO39,KONO55,KODN
700,110.000002,70.0,550.0,550.0,-327.554202,6.247849,-0.000039,0.000090,-0.838868,-0.533825,-241.611215,-241.611215,138.053172
701,110.000002,70.0,550.0,550.0,-327.554202,6.247939,-0.000039,0.000093,-0.838868,-0.533825,-241.611215,-241.611215,138.053172
702,110.000002,70.0,550.0,550.0,-327.554202,6.248030,-0.000040,0.000095,-0.838868,-0.533825,-241.611215,-241.611215,138.053172
703,110.000002,70.0,550.0,550.0,-327.554202,6.248121,-0.000040,0.000098,-0.838868,-0.533825,-241.611215,-241.611215,138.053172
704,110.000002,70.0,550.0,550.0,-327.554202,6.248211,-0.000040,0.000101,-0.838868,-0.533825,-241.611215,-241.611215,138.053172


In [30]:
MTE_elements.loc[720]

IXNO39    110.000002
IXNO55     70.000000
IONO39    550.000000
IONO55    550.000000
IODN     -327.554202
Qx          6.250000
kQD        -0.000072
kQF         0.000170
KXNO39     -0.838868
KXNO55     -0.533825
KONO39   -241.611215
KONO55   -241.611215
KODN      138.053172
Name: 720, dtype: float64